In [46]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

def show_wait_destroy(winname, img):
    cv.imshow(winname, img)
    cv.moveWindow(winname, 500, 0)
    cv.waitKey(0)
    cv.destroyWindow(winname)


In [47]:


mtx=np.loadtxt('cameraMatrix.array')
dist=np.loadtxt('distanceCoefficients.array')
rvecs=np.loadtxt('rotation.array').reshape(1,3,1)
tvecs=np.loadtxt('translation.array').reshape(1,3,1)
img = cv.imread('calibration/undistorted/ch1-1765738034.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
img = cv.undistort(img, mtx, dist, None, newcameramtx)

show_wait_destroy("img",img)


In [48]:
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
show_wait_destroy("gray", gray)

In [49]:
gray = cv.bitwise_not(gray)
bw = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 19, -2)
#show_wait_destroy("binary", bw)
edges = cv.Canny(gray,50,150,apertureSize = 3)
#show_wait_destroy("edges", edges)

In [85]:
def find_intersection_point(p1, p2, p3, p4):
    """
    Finds the intersection point of two infinite lines defined by two points each.

    Args:
        p1, p2: Endpoints of the first line (numpy arrays or lists/tuples of [x, y]).
        p3, p4: Endpoints of the second line.

    Returns:
        A numpy array [x, y] of the intersection point, or None if lines are parallel.
    """
    P = np.array(p1, dtype=float)
    R = np.array(p2, dtype=float) - P
    Q = np.array(p3, dtype=float)
    S = np.array(p4, dtype=float) - Q

    # Construct matrix A and vector B for Ax = B
    A = np.array([
        [R[0], -S[0]],
        [R[1], -S[1]]
    ])
    B = Q - P

    # Calculate the determinant of A
    determinant = np.linalg.det(A)
    
    if np.isclose(determinant, 0):
        # Lines are parallel or collinear
        return None 
    else:
        # Solve for t and u
        tu = np.linalg.solve(A, B)
        t = tu[0]
        # u = tu[1] # u is not needed for the final point
        
        # Calculate intersection point using the first line's equation
        intersection_point = P + t * R
        return intersection_point

In [117]:
#show_wait_destroy("edges", edges)
lines = cv.HoughLines(edges,1,np.pi/180,120)
imgEdges = np.copy(img)
print(np.asarray(imgEdges).shape)
possibleGuages = []
for line in lines:
    rho,theta = line[0]
    if(theta < np.pi/3 or theta > np.pi*2/3):
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv.line(imgEdges,(x1,y1),(x2,y2),(0,0,255),2)
        print(f"{x1},{y1} {x2},{y2}")
        lower_bound = find_intersection_point([x1,y1],[x2,y2],[0,0],[10,0])
        upper_bound = find_intersection_point([x1,y1],[x2,y2],[0,360],[10,360])
        print(f"{int(lower_bound[0])},{int(lower_bound[1])}")
        print(f"{int(upper_bound[0])},{int(upper_bound[1])}")
        if((int(lower_bound[0])<=0 and int(upper_bound[0])<=0) or int(upper_bound[0]) == int(lower_bound[0])):
            continue
        startX=min(lower_bound[0],upper_bound[0])-20
        endX=max(lower_bound[0],upper_bound[0])+20
        possibleGuage=np.copy(img[int(lower_bound[1]):int(upper_bound[1]), max(0,int(startX)):max(0,int(endX))])
        rows,cols,_ = possibleGuage.shape
        rotationAngle=theta*(180/np.pi)
        if(lower_bound[0]<upper_bound[0]):
            rotationAngle=theta*(180/np.pi)+180
        rotationMatrix=cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0),rotationAngle,1)
        possibleGuage=cv.warpAffine(possibleGuage,rotationMatrix,(cols,rows))
        possibleGuages.append(possibleGuage)
show_wait_destroy("imgEdges",imgEdges)
for possibleGuage in possibleGuages:
    possibleGuageEdges = cv.Canny(possibleGuage,50,150,apertureSize = 3)
    show_wait_destroy("possibleGuages",possibleGuageEdges)

(360, 640, 3)
214,-1003 249,995
231,0
237,360
252,-1045 531,935
399,0
449,360
-7,1001 131,-993
62,0
37,360
33,1004 173,-990
103,0
78,360
210,-1045 523,929
375,0
432,360
245,-1037 489,947
372,0
416,360
9,1006 218,-982
114,0
76,360
189,-1011 294,985
242,0
261,360
-40,1002 203,-982
82,0
38,360
19,1005 193,-986
106,0
75,360
-74,998 169,-986
48,0
4,360
241,1004 276,-995
258,0
252,360
21,1000 21,-1000
21,0
21,360
3,1007 246,-977
126,0
82,360
89,1003 158,-995
123,0
111,360


In [15]:
horizontal = np.copy(bw)
vertical = np.copy(bw)
cols = horizontal.shape[1]
horizontal_size = cols // 30
# Create structure element for extracting horizontal lines through morphology operations
horizontalStructure = cv.getStructuringElement(cv.MORPH_RECT, (horizontal_size, 1))
# Apply morphology operations
horizontal = cv.erode(horizontal, horizontalStructure)
horizontal = cv.dilate(horizontal, horizontalStructure)
# Show extracted horizontal lines
show_wait_destroy("horizontal", horizontal)

In [ ]:
rows = vertical.shape[0]
verticalsize = rows // 40
# Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
# Apply morphology operations
vertical = cv2.erode(vertical, verticalStructure)
vertical = cv2.dilate(vertical, verticalStructure)
# Show extracted vertical lines
show_wait_destroy("vertical", vertical)